In [ ]:
# remove all rows that contains missing value 

col_names=['DEPTH', 'DTC', 'DTS', 'CALI', 'DEN', 'DENC', 'GR', 'NEU', 'PEF',
        'RDEP', 'RMED']    


# remove all rows that contains missing value 

df1.dropna(axis=0, subset=col_names, inplace=True)

df1

In [ ]:
#pca = PCA(n_components=116)
corr_matr=df1[1:].corr()
#corr_matr['PHIF'].sort_values(ascending=False)
corr_matr['SW'].sort_values(ascending=False)
#corr_matr['VSH'].sort_values(ascending=False)
#corr_matr
#df1

In [ ]:
# feature selection

# define feature selection
fs = SelectKBest(score_func=f_regression, k=11) #try different score functions
# apply feature selection
X_selected_PHIF = fs.fit_transform(df1.iloc[:,1:13],df1.iloc[:,14])
filter = fs.get_support() #k features taken into account
indexes_best_PHIF=df1.iloc[:,1:13].columns[filter]

X_selected_SW = fs.fit_transform(df1.iloc[:,1:13],df1.iloc[:,15])
filter = fs.get_support() #k features taken into account
indexes_best_SW=df1.iloc[:,1:13].columns[filter]

X_selected_VSH = fs.fit_transform(df1.iloc[:,1:13],df1.iloc[:,16])
filter = fs.get_support() #k features taken into account
indexes_best_VSH=df1.iloc[:,1:13].columns[filter]

indexes_best_PHIF,indexes_best_PHIF,indexes_best_VSH

In [ ]:
#df1_best=df1[indexes_best_PHIF] #activate it you used first option
#clf = IsolationForest(random_state=0).fit_predict(df1_best) #better
#df1_x_train = df1_best[clf==1]
#df1_y_train=df1.iloc[clf==1,[14,15,16]]
#df1_y_train


# identify outliers in the training dataset
#lof = LocalOutlierFactor(random_state=0)  #try other methods    #worse
#lof_pred = lof.fit_predict(df1_best)
#no_outliers = lof_pred != -1


######df1_x_train= df1_best[no_outliers.values, :]


#df1_x_train=df1_best.iloc[no_outliers==True]
#df1_y_train=df1.iloc[no_outliers==True,[14,15,16]]
#df1_y_train

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)

# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

# plot
sc = ax.scatter(x, y, z, s=40, c=x, marker='o', cmap=cmap, alpha=1)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
'''def result_plot(y_predict, y_real, names = ['PHIF', 'SW', 'VSH'], clust_indexes=[],n_points=1000):
    RMSE, R2 = [], []
    print(names)
    for i,name in enumerate(names):
        print(i)
        if isinstance(name, str)==True:
            print(i)
            #print(y_real.iloc[:,i])
            print(y_predict[i])
            RMSE.append(np.sqrt(mean_squared_error(y_real[i], y_predict[i])))
            R2.append(r2_score(y_real[i], y_predict[i]))
        else:
            for j in name:
                print(j)
                RMSE.append(np.sqrt(mean_squared_error(y_real[i], y_predict[i])))
                R2.append(r2_score(y_real[i], y_predict[i]))
                
        
    
    # check the accuracy of predicted data and plot the result
    y_pred_clusts=pd.DataFrame()
    y_true_clusts=pd.DataFrame()
    for i,name in enumerate(names):
        if isinstance(name, str)==True:
            continue
        else:
            y_pred_clust=[]
            y_true_clust=pd.Series([])
            for j in name:
                y_p=y_predict.pop(i)
                y_t=y_real.pop(i)
                y_pred_clust=y_pred_clust+y_p
                y_true_clust=pd.concat([y_true_clust,y_t],axis=0)
                print(f'iter {y_true_clust.size}')
            y_pred_clusts=pd.concat([y_pred_clusts,pd.Series(y_pred_clust)],axis=1)
            y_true_clusts=pd.concat([y_true_clusts,pd.Series(y_true_clust)],axis=1)
            print(f'pred {len(y_pred_clust)},{type(y_p)}')
            print(f'true {len(y_true_clust)},{type(y_t)}')
    y_predict.append(y_pred_clusts,ignore_index=True)
    y_real.append(y_true_clusts,ignore_index=True)
    y_predict=pd.DataFrame(y_predict).T
    y_real=pd.DataFrame(y_true).T
    print(f' {y_real}, {y_predict}')
    print('RMSE:', '{:.5f}'.format(np.sqrt(mean_squared_error(y_real, y_predict))))
    for j in range(len(RMSE)):
        print(f'    {name} : {RMSE[j]:.5f}')
#     print("-"*65)
    
    print( 'R^2: ', r2_score(y_real, y_predict))
    for j in range(len(RMSE)):
        print(f'    {name} : {R2[j]:.5f}')
    
    plt.subplots(nrows=3, ncols=2, figsize=(16,16))

    for i,name in enumerate(names):   
        if isinstance(name, int)==False:           
            plt.subplot(3, 2, i*2+1)
            plt.plot(y_real[:n_points].iloc[:,i])
            plt.plot(y_predict[:n_points, i])
            plt.legend(['True', 'Predicted'])
            plt.xlabel('Sample')
            plt.ylabel(name)
            plt.title(name+' Prediction Comparison')
       
            plt.subplot(3, 2, i*2+2)
            plt.scatter(y_real.iloc[:, i], y_predict[:, i], alpha=0.01)
            plt.xlabel('Real Value')
            plt.ylabel('Predicted Value')
            plt.title(name+' Prediction Comparison')
                   
        else:
            for j in name:
                plt.subplot(3, 2, i*2+1)
                plt.plot(y_real[:n_points].iloc[:,i])
                plt.plot(y_predict[:n_points, i])
                plt.legend(['True', 'Predicted'])
                plt.xlabel('Sample')
                plt.ylabel(name)
                plt.title(name+' Prediction Comparison')

                plt.subplot(3, 2, i*2+2)
                plt.scatter(y_real.iloc[:, i], y_predict[:, i], alpha=0.01)
                plt.xlabel('Real Value')
                plt.ylabel('Predicted Value')
                plt.title(name+' Prediction Comparison')
                   

    plt.show()'''

In [ ]:
# Random forest model
#RF = RandomForestRegressor(n_estimators=100, random_state=100)
#RF_best = grid_search(RF, {},X_train,y_train)
###0.9401138 k=10
### 0.9458 k=9
#result_plot(RF_best.predict(X_train), y_train)
##0.00657 k=11
#0.00715 k=10
# 0.0072 k=9
#RF_best.predict(X_train)
# check the result on validation dataset only
#result_plot(RF_best.predict(X_val), y_val)
##0.09206 k=9
## 0.08295 k=11

#y_pred_clusts=pd.DataFrame()
#y_true_clusts=pd.DataFrame()

#y_pred_clust=[1,2,3]
#y_true_clust=[2,3,4]

#pd.concat([y_pred_clusts,pd.Series(y_pred_clust)],axis=1)
#pd.concat([y_true_clusts,pd.Series(y_true_clust)],axis=1)


In [ ]:
#from xgboost import XGBClassifier
#param_grid=dict(kmeans__n_clusters=range(2,30)) #2 underscores between the estimator
#and the parameters
#knn=KNeighborsClassifier(n_neighbors=25)
#dbscan=DBSCAN(eps=0.05, min_samples=5)
#knn.fit(knn.components_,dbscan.labels_[knn.core_sample_indices_])

#dbscan.fit(X_train)
#reg = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=100))
#pipeline=Pipeline(['model' , Birch(threshold=0.01, n_clusters=n_clust).fit_predict(X_train),
                #   ('reg',MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100,
                    #                                                     random_state=100)))])

#reg = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=100))
#reg = MultiOutputRegressor(XGBClassifier(n_estimators=100, random_state=100))
#reg = MultiOutputRegressor(Ridge(random_state=100))
#reg_best = grid_search(reg, {})
# define the model


'''grid = GridSearchCV(estimator=pipeline, 
                            param_grid=param_grid, 
                            scoring='r2', 
                            cv=5)
    grid.fit(X_train, y_train)
    print(grid.best_score_)

    print(grid.best_estimator_)

    #result_plot(reg_best.predict(X_train), y_train)'''